# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [4]:
# import required libraries
import numpy as np
import pandas as pd
import io
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# your code here
zip_file_path = 'Orders.zip'


with zipfile.ZipFile(zip_file_path, 'r') as z:
    
    print(z.namelist())
    
    csv_filename = z.namelist()[0]
    
    with z.open(csv_filename) as f:
       
        df = pd.read_csv(io.BytesIO(f.read()))

['Orders.csv']


In [6]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [7]:
# your code here
total_spent = df.groupby('CustomerID')['amount_spent'].sum().reset_index()

total_spent.rename(columns={'amount_spent': 'total_spent'}, inplace=True)
total_spent.head()

,CustomerID,total_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [10]:
q95 = total_spent['total_spent'].quantile(0.95)
q75 = total_spent['total_spent'].quantile(0.75)

def determine_status(amount):
    if amount > q95:
        return 'VIP'
    elif q75 < amount <= q95:
        return 'Preferred'
    else:
        return 'Normal'

total_spent['customer_status'] = total_spent['total_spent'].apply(determine_status)
total_spent.head(20)

,CustomerID,total_spent,customer_status
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Normal
5,12352,2506.04,Preferred
6,12353,89.00,Normal
7,12354,1079.40,Normal
8,12355,459.40,Normal
9,12356,2811.43,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [14]:
merged_df = pd.merge(df, total_spent, on='CustomerID')
vip_counts = merged_df[merged_df['customer_status'] == 'VIP']['Country'].value_counts()
vip_counts.head(20)

Country
United Kingdom     84185
EIRE                7077
France              3290
Germany             3127
Netherlands         2080
Australia            898
Portugal             681
Switzerland          594
Spain                511
Norway               420
Channel Islands      364
Finland              294
Cyprus               248
Singapore            222
Japan                205
Sweden               198
Belgium               54
Denmark               36
Name: count, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [15]:
# your code here

vip_preferred_counts = merged_df[merged_df['customer_status'].isin(['VIP', 'Preferred'])]['Country'].value_counts()
vip_preferred_counts.head(20)

Country
United Kingdom     221635
Germany              7349
EIRE                 7238
France               6301
Netherlands          2080
Spain                1569
Belgium              1557
Switzerland          1370
Portugal             1093
Norway               1028
Australia            1028
Channel Islands       589
Italy                 507
Finland               504
Cyprus                451
Sweden                273
Japan                 272
Singapore             222
Denmark               217
Israel                214
Name: count, dtype: int64